In [1]:
import pandas as pd

In [2]:
games = pd.read_parquet('../Datasets/gamesv5.parquet')
reviews = pd.read_parquet('../Datasets/reviews.parquet')
items = pd.read_parquet('../Datasets/itemsv2.parquet')

In [3]:
generos = list(games.drop(columns=['item_id','release_date']).columns)

In [5]:
ids =pd.read_csv('../Datasets/idsnv2.csv')

In [6]:
iid=list(set(items['item_id']))

In [7]:
lanio=list(set(reviews['posted'].to_list()))

In [8]:
def PlayTimeGenre(genero):
  genero=genero.upper()
  if genero in generos:
    lgames = games.loc[games[genero] == 1, ['release_date', 'item_id']]
    lhoras = items.groupby('item_id')['playtime_forever'].sum().reset_index()
    dfplay = pd.merge(lgames, lhoras, on='item_id', how='inner')
    df_horas = dfplay.groupby('release_date')['playtime_forever'].sum().reset_index()
    df_horas_ordenado = df_horas.sort_values(by='playtime_forever', ascending=False)
    resultado=df_horas_ordenado.head(1)
    prin = "Año de lanzamiento con más horas jugadas para Género " + genero
    anio = int(resultado['release_date'].iloc[0])
    return {prin : anio}
  else:
    return {"El Genero indicado no se encuentra en la base de datos":genero}

In [9]:
PlayTimeGenre("2d")

{'Año de lanzamiento con más horas jugadas para Género 2D': 2011}

In [10]:
def UserForGenre(genero):
  genero=genero.upper()
  if genero in generos:
    dfgames = games[['item_id','release_date',genero]]
    dfgames = dfgames[games[genero]==1]
    dfuserg = pd.merge(dfgames,items,on='item_id',how='inner')
    uhoras = dfuserg.groupby('user_id')['playtime_forever'].sum().reset_index()
    uhoras = uhoras.sort_values(by='playtime_forever', ascending=False)
    uhoras = uhoras.head(1)
    uhoras = uhoras['user_id'].iloc[0]
    dfuserg = dfuserg[dfuserg['user_id']==uhoras]
    dfuserg = dfuserg.groupby('release_date')['playtime_forever'].sum().reset_index()
    dfuserg = dfuserg.sort_values(by='release_date', ascending=False)
    dfuserg = dfuserg[dfuserg['playtime_forever']>0]
    dfuserg = dfuserg.astype(int)
    dfuserg.rename(columns={'release_date': 'Año', 'playtime_forever':'Horas'}, inplace=True)
    di = dfuserg.apply(lambda row: row.to_dict(), axis=1).tolist()
    prin="Usuario con más horas jugadas para Género " + genero
    return {prin:uhoras,"Horas Jugadas":di}
  else:
    return {"El Genero indicado no se encuentra en la base de datos ":genero}

In [11]:
UserForGenre('AcTiOn')

{'Usuario con más horas jugadas para Género ACTION': 'Sp3ctre',
 'Horas Jugadas': [{'Año': 2017, 'Horas': 43327},
  {'Año': 2016, 'Horas': 30195},
  {'Año': 2015, 'Horas': 394314},
  {'Año': 2014, 'Horas': 132864},
  {'Año': 2013, 'Horas': 122158},
  {'Año': 2012, 'Horas': 384238},
  {'Año': 2011, 'Horas': 155459},
  {'Año': 2010, 'Horas': 82766},
  {'Año': 2009, 'Horas': 108771},
  {'Año': 2008, 'Horas': 8533},
  {'Año': 2007, 'Horas': 112794},
  {'Año': 2006, 'Horas': 90253},
  {'Año': 2005, 'Horas': 21339},
  {'Año': 2004, 'Horas': 127411},
  {'Año': 2003, 'Horas': 7744},
  {'Año': 2002, 'Horas': 238},
  {'Año': 2001, 'Horas': 250},
  {'Año': 2000, 'Horas': 70644},
  {'Año': 1999, 'Horas': 44},
  {'Año': 1997, 'Horas': 242},
  {'Año': 1995, 'Horas': 217}]}

In [12]:
def UsersRecommend(anio):
  if anio in lanio:
    dfa = reviews[(reviews['posted'] == anio) & (reviews['recommend'] == True) & (reviews['sentiment_analysis'] >0)].copy()
    dfa = dfa.groupby('item_id')['sentiment_analysis'].sum().reset_index()
    dfa = dfa.sort_values('sentiment_analysis', ascending=False)
    top_games = dfa.head(3)['item_id'].to_list()
    for i,g in enumerate(top_games):
      if g in iid:
        top_games[i]=items[items['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
      else:
        top_games[i]=ids[ids['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
    return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':top_games[2]}
  else:
    return {f'El año indicado no se encuentra en la base de datos: ':anio}

In [13]:
UsersRecommend(2012)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': 'Team Fortress 2',
 'Puesto 3': 'Borderlands 2'}

In [14]:
def UsersNotRecommend(anio):
  if anio in lanio:
    dfa = reviews[(reviews['posted'] == anio) & (reviews['recommend'] == False) & (reviews['sentiment_analysis'] ==0)].copy()
    dfa['sentiment_analysis']=1
    dfa = dfa.groupby('item_id')['sentiment_analysis'].sum().reset_index()
    dfa = dfa.sort_values('sentiment_analysis', ascending=False)
    top_games = dfa.head(3)['item_id'].to_list()
    for i,g in enumerate(top_games):
      if g in iid:
        top_games[i]=items[items['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
      else:
        top_games[i]=ids[ids['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
    if len(top_games)<2:
      return{'Puesto 1':top_games[0],'Puesto 2':'','Puesto 3':''}
    elif len(top_games)<3:
      return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':''}
    else:
      return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':top_games[2]}
  else:
    return {f'El año indicado no se encuentra en la base de datos: ':anio}

In [15]:
UsersNotRecommend(2012)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': 'Ace of Spades',
 'Puesto 3': 'Chivalry: Medieval Warfare'}

In [16]:
def sentiment_analysis(anio):
  dfs = pd.merge(reviews[['item_id','sentiment_analysis']],games[['item_id','release_date']].drop_duplicates(),on='item_id',how='inner')
  dfs = dfs[dfs['release_date']==anio]
  dfs = dfs.groupby('sentiment_analysis').size().reset_index(name='count')
  dfs = dfs.sort_values('sentiment_analysis', ascending=True)
  dfs = dfs['count'].to_list()
  return{'Negative':dfs[0],'Neutral':dfs[1],'Positive':dfs[2]}

In [17]:
sentiment_analysis(2014)

{'Negative': 1040, 'Neutral': 1067, 'Positive': 4341}